# KenLM Sentence-base

#### based on:

https://kheafield.com/papers/avenue/kenlm.pdf 

https://kheafield.com/papers/edinburgh/estimate_paper.pdf

#### implementation of the estimation part:

https://kheafield.com/code/kenlm/

Language models are estimated from text using __[modified](http://www.ee.columbia.edu/~stanchen/papers/h015a-techreport.pdf)__ [Kneser-Ney smoothing](https://ieeexplore.ieee.org/document/479394) without pruning. It is done on disk, enabling one to build much larger models. Kneser-Ney smothng consistently outperforms all other n-grams models with smothing evaluated in this [techreport](http://www.ee.columbia.edu/~stanchen/papers/h015a-techreport.pdf) by Chen and Goodman.

In this notebook I am splitting and tokenizing __[Ukrainian Brown Corpus](https://github.com/brown-uk/corpu)__ (good and so-so datasets) into sentences including symbols. Then I use KenLM scripts to estimate ARPA n-gram sentense-based models. KenLM script __lmplz__ by default uses $<s>$ and $</s>$ tags at the beginning and end of each sentence. 

It is only sentence-based model and it estimates score for each sentence separately. The score is equal to the  log10 probability of the sentence. Then I sum up scores for all the sentences in the corpus, divide them by the number of the words in the corpus and take 10 to the power of the resulting fraction to calculate the __perplexity__ of my model.

## Installing KenLM

In [3]:
import os
import kenlm

    ---# sudo apt-get install build-essential libboost-all-dev cmake zlib1g-dev libbz2-dev liblzma-dev 
    
    ---# wget -O - https://kheafield.com/code/kenlm.tar.gz |tar xz
         mkdir -p build && cd build
         cmake ..
         make -j 4
         
## Tokenize text
         
Here the __[Ukrainian Brown Corpus](https://github.com/brown-uk/corpu)__ (good and so-so datasets) is splitted into sentences including symbols.
         
#### [TokenizeText.groovy](https://github.com/brown-uk/nlp_uk/blob/master/src/main/groovy/org/nlp_uk/tools/README.md)


Аналізує текст і записує результат у виходовий файл:

        розбиває на речення (-s)
        розбиває на токени (-w) (результати включають пунктуацію тому всі токени розділяються вертикальними рисками)
        розбиває на слова (-u)


In [2]:
!groovy nlp_uk/nlp_uk/src/main/groovy/org/nlp_uk/tools/TokenizeText.groovy -s -w -i brown-uk/corpus/final_all_GS_tagged.txt -o brown-uk/corpus/final_all_GS_tagged_words_symbols_sentences.txt

writing into brown-uk/corpus/final_all_GS_tagged_words_symbols_sentences.txt


_!!! Should I lowercase? !!!_

In [3]:
import fileinput
filename=os.path.join(os.path.abspath(''), 'brown-uk','corpus','final_all_GS_tagged_words_symbols_sentences.txt')
for line in fileinput.FileInput(filename, inplace=1):
        #line='<s>'+line
        line=line.replace("|"," ")#.lower()
        line=line.replace("\n","")
        line=line.replace(r"\n","")
        line=line.replace("_foreign_"," _foreign_ ")
        #line=line.replace("BEGIN_TEXT","")
        #line=line.replace("END_TEXT","")
        #line=line.replace("\n","</s>")
        #line=line.replace(r"<s>\n","BEGIN <s>")
        #line=line.replace("<s></s>","")
        #line=line.replace(r"\n </s>","</s> END")
        #line=line.replace("<s>","")
        print (line)
#for line in fileinput.FileInput(filename, inplace=1):
        #line=line.replace("\n","")
        #print (line)

Example result file: (+lowercase)

    У   2013   році ,   до   100-річчя   виходу   першого   числа   журналу   _FOREIGN_ ,   на   будинку   встановили   меморіальну   дошку .   
    Тоді   ж   таки   в   будинку   відбувся   перший   з'їзд   есперантистів ,   на   якому   було   50   есперантистів   з   усієї   України   і   троє   з-за   кордону .   
    Відтоді   щороку   вони   там   організовують   конференції ,   починаючи   з   2013-го .   
    Щороку   там   вручають   премію   тим ,   хто   пропаґує   український   есперантський   рух   та   український   погляд   на   важливі   події .  
    _FOREIGN_  Більшість   дописів   на   сторінці   Сергія   Шматкова   у   соціальній   мережі   _FOREIGN_   —   мовою   есперанто .  
    « У   мене   у   _FOREIGN_   понад   дві   тисячі   друзів   з   усього   світу ,   з   якими   я   спілкуюсь   мовою   есперанто » ,   —   розповідає   пан   Сергій .  
    Цю   незвичну   для   багатьох   мову   Сергій   Шматков   вивчив   ще   у   1980-х .   
    Народився   і   прожив   чоловік   усе   життя   в   Донецькій   області ,   а   після   окупації   перебрався   до   Львова .  
    
### Number of sentences

In [4]:
with open("brown-uk/corpus/final_all_GS_tagged_words_symbols_sentences.txt", "r") as input:
    summa=0
    for line in input:
        summa+=1
summa

39613

## Estimating Large Language Models with KenLM

Tokenized and splitted into sentences __[Ukrainian Brown Corpus](https://github.com/brown-uk/corpu)__ (good and so-so datasets) is provided on stdin and the __ARPA__ is written to stdout.

#### kenlm/build/bin/lmplz -o -S -T    
        -o
            Required. Order of the language model to estimate.
        -S
            Recommended. Memory to use. This is a number followed by single-character suffix: % for percentage of physical memory (on platforms where this is measured), b for bytes, K for kilobytes, M for megabytes, and so on for G and T. If no suffix is given, kilobytes are assumed for compatability with GNU sort. The sort program is not used; the command line is simply designed to be compatible.
        -T
            Recommended. Temporary file location.

Here 3-gram, 4-gram, 5-gram and 6-gram models are estimated by kenlm library and saved into the propriate ARPA files.
            

In [5]:
!kenlm/build/bin/lmplz -o 3 -S 10% <brown-uk/corpus/final_all_GS_tagged_words_symbols_sentences.txt> brown-uk/corpus/kenlm/final_all_GS_tagged_words_symbols_sentences_based_3.arpa
!kenlm/build/bin/lmplz -o 4 -S 10% <brown-uk/corpus/final_all_GS_tagged_words_symbols_sentences.txt> brown-uk/corpus/kenlm/final_all_GS_tagged_words_symbols_sentences_based_4.arpa
!kenlm/build/bin/lmplz -o 5 -S 10% <brown-uk/corpus/final_all_GS_tagged_words_symbols_sentences.txt> brown-uk/corpus/kenlm/final_all_GS_tagged_words_symbols_sentences_based_5.arpa
!kenlm/build/bin/lmplz -o 6 -S 10% <brown-uk/corpus/final_all_GS_tagged_words_symbols_sentences.txt> brown-uk/corpus/kenlm/final_all_GS_tagged_words_symbols_sentences_based_6.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /home/ana/Downloads/master diploma/code/brown-uk/corpus/final_all_GS_tagged_words_symbols_sentences.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 732089 types 103347
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:1240164 2:580684032 3:1088782592
Statistics:
1 103347 D1=0.669311 D2=1.10758 D3+=1.48587
2 449944 D1=0.856159 D2=1.22555 D3+=1.45964
3 635398 D1=0.923246 D2=1.35143 D3+=1.41611
Memory estimate for binary LM:
type       kB
probing 24338 assuming -p 1.5
probing 27378 assuming -r models -p 1.5
trie    11637 without quantization
trie     7275 assuming -q 8 -b 8 quantization 
trie    10946 assuming -a 22 array pointer compression
trie     6583 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting 

In [4]:
LM3 = os.path.join(os.path.abspath(''), 'brown-uk', 'corpus', 'kenlm', 'final_all_GS_tagged_words_symbols_sentences_based_3.arpa')
LM4 = os.path.join(os.path.abspath(''), 'brown-uk', 'corpus', 'kenlm', 'final_all_GS_tagged_words_symbols_sentences_based_4.arpa')
LM5 = os.path.join(os.path.abspath(''), 'brown-uk', 'corpus', 'kenlm', 'final_all_GS_tagged_words_symbols_sentences_based_5.arpa')
LM6 = os.path.join(os.path.abspath(''), 'brown-uk', 'corpus', 'kenlm', 'final_all_GS_tagged_words_symbols_sentences_based_6.arpa')

In [5]:
model3 = kenlm.LanguageModel(LM3)
model4 = kenlm.LanguageModel(LM4)
model5 = kenlm.LanguageModel(LM5)
model6 = kenlm.LanguageModel(LM6)

### Sentences scores

#### model.score(self, sentence, bos = True, eos = True)

Return the __log10 probability of a string__.  By default, the string is treated as a sentence.  
          
          return log10 p(sentence </s> | <s>)

If you do not want to condition on the beginning of sentence, pass __bos = False__ . Never include $<s>$ as part of the string. 

Similarly, the end of sentence token $</s>$ can be omitted with __eos = False__. Since language models explicitly predict $</s>$, it can be part of the string.


I do not use bos or eos = False, so the method calculates scores of those strings to be sentences.

In [17]:
sentence1 = 'Штучний інтелект врятує світ .'
print(sentence1)
print('{0}-gram model'.format(model3.order))
print(model3.score(sentence1))
print('{0}-gram model'.format(model4.order))
print(model4.score(sentence1))
print('{0}-gram model'.format(model5.order))
print(model5.score(sentence1))
print('{0}-gram model'.format(model6.order))
print(model6.score(sentence1))

Штучний інтелект врятує світ .
3-gram model
-19.345046997070312
4-gram model
-19.335840225219727
5-gram model
-19.335840225219727
6-gram model
-19.335840225219727


In [18]:
sentence2 = '_#foreign_ врятує світ .'
print(sentence2)
print('{0}-gram model'.format(model3.order))
print(model3.score(sentence2))
print('{0}-gram model'.format(model4.order))
print(model4.score(sentence2))
print('{0}-gram model'.format(model5.order))
print(model5.score(sentence2))
print('{0}-gram model'.format(model6.order))
print(model6.score(sentence2))

_#foreign_ врятує світ .
3-gram model
-11.154338836669922
4-gram model
-11.141000747680664
5-gram model
-11.141000747680664
6-gram model
-11.141000747680664


In [19]:
sentence3 = 'Наука врятує світ .'
print(sentence3)
print('{0}-gram model'.format(model3.order))
print(model3.score(sentence3))
print('{0}-gram model'.format(model4.order))
print(model4.score(sentence3))
print('{0}-gram model'.format(model5.order))
print(model5.score(sentence3))
print('{0}-gram model'.format(model6.order))
print(model6.score(sentence3))

Наука врятує світ .
3-gram model
-12.354094505310059
4-gram model
-12.335543632507324
5-gram model
-12.335543632507324
6-gram model
-12.335543632507324


In [20]:
sentence4 = 'Краса врятує світ .'
print(sentence4)
print('{0}-gram model'.format(model3.order))
print(model3.score(sentence4))
print('{0}-gram model'.format(model4.order))
print(model4.score(sentence4))
print('{0}-gram model'.format(model5.order))
print(model5.score(sentence4))
print('{0}-gram model'.format(model6.order))
print(model6.score(sentence4))

Краса врятує світ .
3-gram model
-13.028851509094238
4-gram model
-13.010300636291504
5-gram model
-13.010300636291504
6-gram model
-13.010300636291504


#### Check that total full score = direct score

In [21]:
def score(model,s):
    return sum(prob for prob, _, _ in model.full_scores(s))

In [22]:
assert (abs(score(model6, sentence1) - model6.score(sentence1)) < 1e-3)
assert (abs(score(model6, sentence2) - model6.score(sentence2)) < 1e-3)
assert (abs(score(model6, sentence3) - model6.score(sentence3)) < 1e-3)
assert (abs(score(model6, sentence4) - model6.score(sentence4)) < 1e-3)

#### Show scores and n-gram matches

In [25]:
words = ['<s>'] + sentence1.split() + ['</s>']
for i, (prob, length, oov) in enumerate(model6.full_scores(sentence4)):
    print('{0} {1}: {2}'.format(prob, length, ' '.join(words[i+2-length:i+2])))
    if oov:
        print('\t"{0}" is an OOV'.format(words[i+1]))

-5.369656085968018 2: <s> Штучний
-5.358176231384277 1: інтелект
-1.317441701889038 2: інтелект врятує
-0.9645605683326721 2: врятує світ
-0.0004651869530789554 3: врятує світ .


__________________________________________

    -4.6759634	краса	-0.26173612
    -1.8974496	краса врятує	-0.026417483
    -0.97120434	краса врятує світ	-0.00874679
    -4.032551	</s>	0



Чому n-gramm які закінчуютьмя на $</s>$ завжди друга цифра нуль. Виходить нема залежності між реченнями. Як на мене це не ок.

#### Calculating the perplexity of the sentence

In [26]:
def perplexity(model, sentence, bos=True, eos=True):
    """
    Compute perplexity of a sentence.
    @param sentence One full sentence to score.  Do not include <s> or </s>.
    """
    words = len(str(sentence).split()) + 1 # For </s>
    return 10.0**(-model.score(sentence, bos=bos, eos=eos) / words)

In [27]:
print(perplexity(model6, sentence4))

400.0001251964878


In [28]:
print(model6.perplexity(sentence4))

400.0001251964878


#### Find out-of-vocabulary words

In [29]:
for w in words:
    if not w in model6:
        print('"{0}" is an OOV'.format(w))

"Штучний" is an OOV


### Calculating the perplexity of the model on the Ukrainian brown corpus (good and so-so)

_!!! keep in mind this is the same dataset I made my estimation ARPA models on !!!_

In [30]:
filename=os.path.join(os.path.abspath(''), 
                      'brown-uk', 'corpus',
                      'final_all_GS_tagged_words_symbols_sentences.txt')

#read sentence by sentence
temp = open(filename,'r').read().split('\n')

It is only sentence-based model and it estimates score for each sentence separately. The score is equal to the  log10 probability of the sentence. Then I sum up scores for all the sentences in the corpus, divide them by the number of the words in the corpus and take 10 to the power of the resulting fraction to calculate the __perplexity__ of my model.

In [31]:
def perplexity_on_texts_by_sentences(model, temp):
    all_score=0
    all_words=0
    for sentence in temp:
        all_score+=model.score(sentence, bos = False, eos = False)
        all_words+=len(str(sentence).split())
    print("all_score: "+str(all_score)+"; \nnumber of tokens in text: "+str(all_words))
    return 10.0**(-all_score / all_words)

In [32]:
print('{0}-gram model'.format(model3.order))
print('perplexity: ', str(perplexity_on_texts_by_sentences(model3,temp))+"\n")
print('{0}-gram model'.format(model4.order))
print('perplexity: ',str(perplexity_on_texts_by_sentences(model4,temp))+"\n")
print('{0}-gram model'.format(model5.order))
print('perplexity: ',str(perplexity_on_texts_by_sentences(model5,temp))+"\n")
print('{0}-gram model'.format(model6.order))
print('perplexity: ',str(perplexity_on_texts_by_sentences(model6,temp))+"\n")

3-gram model
all_score: -1062891.6325109005; 
number of tokens in text: 732044
perplexity:  28.310691259369687

4-gram model
all_score: -935254.2013456821; 
number of tokens in text: 732044
perplexity:  18.949287228877438

5-gram model
all_score: -912237.1490190029; 
number of tokens in text: 732044
perplexity:  17.625875906382273

6-gram model
all_score: -908319.2384450436; 
number of tokens in text: 732044
perplexity:  17.40999686988621



### Perplexity including BOS and EOS tags. Copied their function but rewriten to match text

In [33]:
def perplexity_on_texts_by_sentences_boseos(model, temp):
    all_score=0
    all_words=0
    for sentence in temp:
        all_score+=model.score(sentence)
        all_words+=len(str(sentence).split())+1
    print("all_score: "+str(all_score)+"; \nnumber of tokens in text: "+str(all_words))
    return 10.0**(-all_score / all_words)

In [34]:
print('{0}-gram model'.format(model3.order))
print(str(perplexity_on_texts_by_sentences_boseos(model3,temp))+"\n")
print('{0}-gram model'.format(model4.order))
print(str(perplexity_on_texts_by_sentences_boseos(model4,temp))+"\n")
print('{0}-gram model'.format(model5.order))
print(str(perplexity_on_texts_by_sentences_boseos(model5,temp))+"\n")
print('{0}-gram model'.format(model6.order))
print(str(perplexity_on_texts_by_sentences_boseos(model6,temp))+"\n")

3-gram model
all_score: -981034.0454454422; 
number of tokens in text: 771658
18.678099985577454

4-gram model
all_score: -847061.5530700684; 
number of tokens in text: 771658
12.523227928866882

5-gram model
all_score: -821626.6746518612; 
number of tokens in text: 771658
11.607934511597323

6-gram model
all_score: -816845.5673232079; 
number of tokens in text: 771658
11.4435049092108

